# imports:

In [1]:
from sklearn.svm import SVR as svr
from sklearn.svm import NuSVR
from sklearn.ensemble import ExtraTreesRegressor as etr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.neighbors import KNeighborsRegressor as Kneigh
from sklearn.ensemble import AdaBoostRegressor as Ada
from sklearn.ensemble import HistGradientBoostingRegressor as HG
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import uproot
import pickle
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load events

In [2]:
events_P = pd.read_pickle("run3_1to11.pkl")


In [3]:
a = 0
len_e = range(len(events_P))
for i in len_e:
    #print(i)
    if (events_P['true'].iloc[i] == 0):
        #print(events_P['true'].iloc[i])
        events_P = events_P.drop(events_P.index[i])
        a += 1
    if a + i == len(len_e) - 1: 
        break
        
        

In [4]:
a = 0
len_e = range(len(events_P))
for i in len_e:
    #print(i)
    if (events_P['true'].iloc[i] == 0):
        #print(events_P['true'].iloc[i])
        events_P = events_P.drop(events_P.index[i])
        a += 1
    if a + i == len(len_e) - 1: 
        break

In [5]:
events_P = events_P.set_axis(range(len(events_P)), axis='index')
events_P

,true,p,ecal,hcal,ho,eta,phi,charge,dr,Eecal,...,pfcID,pfcs,correcal,corrhcal,Ccorrecal,Ccorrhcal,run,evt,lumiBlock,time
0,73.406746,70.534348,0.000000,39.231571,0.0,1.789823,0.807020,-1,[],[],...,[],[1],[],[],0.000000,38.852215,1,15538004,15539,4
1,78.327156,79.441994,0.401830,106.128387,0.0,0.589694,-1.123832,-1,[],[],...,[],[1],[],[],1.097574,79.441994,1,15538003,15539,4
2,112.504486,114.411736,82.177238,15.069984,0.0,1.607347,-2.664334,-1,[],[],...,[],[1],[],[],110.778938,18.735685,1,15538007,15539,4
3,151.854996,153.601181,77.562042,1.805020,0.0,1.733131,-1.923388,-1,[],[],...,[],[1],[],[],120.345459,4.415516,1,15538006,15539,4
4,71.967720,70.927727,4.590099,65.932861,0.0,1.956307,1.061470,-1,[],[],...,[],[1],[],[],9.200743,75.419907,1,15538008,15539,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90998,26.848831,25.834078,0.000000,7.537361,0.0,-2.465314,0.847006,-1,[],[],...,[],[1],[],[],0.000000,9.554542,1,7777997,7778,2
90999,67.420471,66.796791,38.099625,12.795702,0.0,0.342099,0.561426,-1,[],[],...,[],[1],[],[],60.096107,15.761148,1,7777992,7778,2
91000,93.527283,94.532738,0.000000,108.618713,0.0,-0.496705,0.886237,-1,[],[],...,[],[1],[],[],0.000000,104.772881,1,7777995,7778,2
91001,170.652069,169.286804,1.452716,223.973984,0.0,-1.565183,1.667673,-1,[],[],...,[],[1],[],[],1.258437,169.286804,1,7778000,7778,2


In [6]:
events_P = events_P.sample(90000)

# get regression input variables and target variable:

# split data:

In [7]:
eraw =  events_P['ecal']+ events_P['hcal']

In [8]:
ecalDivRaw = eraw/events_P['true']
#ecalDivRaw = np.array(ecalDivRaw)

In [9]:
ecalDivRaw

9064     0.873754
17585    1.053124
21410    1.114098
69701    1.011281
54701    1.130995
           ...   
54215    0.535573
71266    0.878893
24823    1.060951
81329    1.246925
53149    1.052683
Length: 90000, dtype: float32

In [11]:

regVars = events_P[['eta','phi','p']]
regVars['ecalDivRaw'] = ecalDivRaw
tgtVar = eraw/events_P['true']
#np.array((eraw - events_P['true'])/events_P['true'])

C:\Users\jonla\AppData\Local\Temp\ipykernel_17588\1395484632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regVars['ecalDivRaw'] = ecalDivRaw


In [12]:
xtrain, xtest, ytrain, ytest, events_train, events_test = train_test_split(regVars, tgtVar,events_P, test_size=0.15)
ytrain = np.ravel(ytrain)

# run models:

In [13]:
#xgbr
XGBR = XGBRegressor(verbosity = 0).fit(xtrain, ytrain)

#ETR
ETR = etr(n_estimators=100, random_state=0).fit(xtrain, ytrain) 

#nusvr
#NSVR_ZS = NuSVR(C=1.0, cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
#      max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=False).fit(xtrain_ZS, ytrain_ZS)

#svr
#SVR_ZS = svr(C=1.0, epsilon=0.2).fit(xtrain_ZS, ytrain_ZS)


#HistGBR
HGBR = HG().fit(xtrain, ytrain)

#ADA
ADA = Ada(random_state=0, n_estimators=100).fit(xtrain, ytrain)

#KNR
KNR = Kneigh(n_neighbors=2).fit(xtrain, ytrain)

#RFR
RFR = rfr(max_depth=2, random_state=0).fit(xtrain, ytrain)

In [14]:

print(ETR.score(xtest,ytest))

print(XGBR.score(xtest,ytest))
print(ETR.predict(xtest))
ytest

0.99999855274627
0.9999781997330128
[1.0936742  0.79882042 1.36513717 ... 1.11528418 0.90282558 0.69611236]


73030    1.093685
81769    0.798821
82522    1.365065
21168    0.739882
20644    0.871821
           ...   
72849    0.870699
1329     1.178694
52200    1.115271
23591    0.902828
58107    0.696100
Length: 13500, dtype: float32

# save models:

In [15]:
pickle.dump(XGBR, open('XGBR_model_run3_1to11.sav', 'wb'))
pickle.dump(ETR, open('ETR_model_run3_1to11.sav', 'wb'))
#pickle.dump(NSVR_ZS, open('NSVR_model_ZS.sav', 'wb'))
#pickle.dump(SVR_ZS, open('SVR_model_ZS.sav', 'wb'))
pickle.dump(HGBR, open('HGBR_model_run3_1to11.sav', 'wb'))
pickle.dump(ADA, open('ADA_model_run3_1to11.sav', 'wb'))
pickle.dump(KNR, open('KNR_model_run3_1to11.sav', 'wb'))
pickle.dump(RFR, open('RFR_model_run3_1to11.sav', 'wb'))

In [16]:
xtest.to_pickle("xtest_run3_1to11.pkl")
ytest.to_pickle("ytest_run3_1to11.pkl")
events_test.to_pickle("events_test_run3_1to11.pkl")